## Import Library

In [12]:
from tensorflow.keras import callbacks
from yolov4.tf import YOLOv4, YOLODataset, SaveWeightsCallback
import pymysql
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm

Call tf.config.experimental.set_memory_growth(GPU0, True)


## DB불러오기(S1에 해당하는 데이터)

In [13]:
conn = pymysql.connect(host='192.168.0.53', user='mingsu', password='mingsu', port=3306, database='humanf',
                       charset='utf8', cursorclass=pymysql.cursors.DictCursor)

sql = """
select
    concat(adit2.IMG_PATH , adat.IMG_NAME) as IMG_PATH,
    (
        select
            group_concat((concat(act.CLASS_CODE, ',', A.COOR)) separator ' ') as ST
        from
        (
            select
                A.IMG_NAME,
                concat(
                    round((A.x1+A.x2)/2/A.IMG_WIDTH, 6),
                    ',',
                    round((A.y1+A.y2)/2/A.IMG_HEIGHT,6),
                    ',',
                    round(abs(A.x1-A.x2)/A.IMG_WIDTH, 6),
                    ',',
                    round(abs(A.y1-A.y2)/A.IMG_HEIGHT, 6)) as COOR
            from
            (
                select
                    adit.IMG_PATH,
                    substring_index(replace(replace(adit.RESOLUTION, '[', ''), ']', ''), ',', 1) as IMG_WIDTH,
                    substring_index(substring_index(replace(replace(adit.RESOLUTION, '[', ''), ']', ''), ',', -1), ',', -1) as IMG_HEIGHT,
                    A.*,
                    substring_index(A.COORDINATES, ',' , 1) as x1,
                    substring_index(substring_index(A.COORDINATES, ',' , 2), ',', -1) as y1,
                    substring_index(substring_index(substring_index(A.COORDINATES, ',' , 3), ',', -1), ',',-1) as x2,
                    substring_index(substring_index(substring_index(substring_index(A.COORDINATES, ',' , 4), ',', -1), ',',-1), ',', -1) as y2
                from
                 (
                    select
                        adat.IMG_NAME,
                        adat.CLASS,
                        replace(adat.COORDINATES, '|', '') as COORDINATES
                    from
                        ai_data_annotation_tbl adat
                     where
                           adat.IMG_NAME like 'S1%'
                 ) A
                left outer join
                     ai_data_img_tbl adit
                on
                    A.IMG_NAME = adit.IMG_NAME
            ) A
        ) A
        where
            A.IMG_NAME = adat.IMG_NAME
    ) as COORS
from
     ai_data_annotation_tbl adat
left outer join
    ai_class_tbl act
on
    adat.CLASS = act.CLASS_NAME
left outer join
    ai_data_img_tbl adit2
on
    adat.IMG_NAME = adit2.IMG_NAME
where
      adat.IMG_NAME like 'S1%'
group by
    adat.IMG_NAME
order by rand() limit 10000;
"""
# sql = """
#     select
#        concat(adit.IMG_PATH ,adat.IMG_NAME) as IMG_PATH,
#        (
#            select group_concat((concat(replace(a2.COORDINATES, '|', ''), ',' ,act.CLASS_CODE)) separator ' ') as st from ai_data_annotation_tbl a2 where a2.IMG_NAME = adat.IMG_NAME
#            ) as COORS
# from
#     ai_data_annotation_tbl adat
#     left outer join
#         ai_class_tbl act
# on
#     adat.CLASS = act.CLASS_NAME
#     left join
#         ai_data_img_tbl adit
#     on
#         adat.IMG_NAME = adit.IMG_NAME
#
# where adat.IMG_NAME like 'S1%'
# group by
#     adat.IMG_NAME
# order by rand() limit 10000;
# """
air_df = pd.read_sql(sql, conn)

In [14]:
X = []
for i in range(len(air_df)):
    X.append(air_df.loc[i, 'IMG_PATH'] + ' ' + air_df.loc[i, 'COORS'])
X
train_ratio = 0.7
validation_ratio = 0.3
# test_ratio = 0.7

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
# stratify= 정답값(Classification)
x_train, x_val = train_test_split(X, test_size=validation_ratio, shuffle=True, random_state = 42)


In [25]:
from tensorflow.keras import callbacks
from yolov4.tf import YOLOv4, YOLODataset, SaveWeightsCallback

yolo = YOLOv4()

# class name
yolo.config.parse_names("data/classes/class_name.txt")
# Context-Free Grammar
yolo.config.parse_cfg("yolov4.cfg")

yolo.make_model()
# load pre_trained weights
yolo.load_weights(
    "yolov4.weights",
    weights_type="yolo",
)
yolo.summary(summary_type="yolo")

for i in range(29):
    yolo.model.get_layer(index=i).trainable = False

yolo.summary()

# Train_set
# image class,x,y,w,h class,x,y,w,h class,x,y,w,h ....
train_dataset = YOLODataset(
    config=yolo.config,
    dataset_list=x_train,
    # If you want to directory,
    # image_path_prefix="/content/train2017",
    training=True,
)

# Val_set
val_dataset = YOLODataset(
    config=yolo.config,
    dataset_list=x_val,
    # image_path_prefix="/content/val2017",
    training=False,
)

yolo.compile()

_callbacks = [
    callbacks.TerminateOnNaN(),
    callbacks.TensorBoard(
        # Directory for tensorBoard
        log_dir="air_df/",
        update_freq=200,
        histogram_freq=1,
    ),
    SaveWeightsCallback(
        yolo=yolo,
        dir_path="air_df/",
        weights_type="yolo",
        step_per_save=2000,
    ),
]

yolo.fit(
    train_dataset,
    callbacks=_callbacks,
    validation_data=val_dataset,
    verbose=3  # 3: print step info
)

ValueError: Memory growth cannot differ between GPU devices

In [1]:
import tensorflow as tf

tf.config.experimental.list_physical_devices('GPU')




In [2]:
#================================================================
#
#   File name   : train.py
#   Author      : PyLessons
#   Created date: 2020-08-06
#   Website     : https://pylessons.com/
#   GitHub      : https://github.com/pythonlessons/TensorFlow-2.x-YOLOv3
#   Description : used to train custom object detector
#
#================================================================
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
from tensorflow.python.client import device_lib
import shutil
import numpy as np
import tensorflow as tf
#from tensorflow.keras.utils import plot_model
from yolov3.dataset import Dataset
from yolov3.yolov4 import Create_Yolo, compute_loss
from yolov3.utils import load_yolo_weights
from yolov3.configs import *
from evaluate_mAP import get_mAP


if YOLO_TYPE == "yolov4":
    Darknet_weights = YOLO_V4_TINY_WEIGHTS if TRAIN_YOLO_TINY else YOLO_V4_WEIGHTS
if YOLO_TYPE == "yolov3":
    Darknet_weights = YOLO_V3_TINY_WEIGHTS if TRAIN_YOLO_TINY else YOLO_V3_WEIGHTS
if TRAIN_YOLO_TINY: TRAIN_MODEL_NAME += "_Tiny"


def main():
    global TRAIN_FROM_CHECKPOINT

    # gpus = tf.config.experimental.list_physical_devices('GPU')
    physical_devices = tf.config.experimental.list_physical_devices('GPU')
    print(f'GPUs {physical_devices}')
    if len(physical_devices) > 0:
        try:
            tf.config.set_visible_devices(physical_devices[0:1], 'GPU')
            logical_devices = tf.config.list_logical_devices('GPU')
        except RuntimeError: pass

    if os.path.exists(TRAIN_LOGDIR): shutil.rmtree(TRAIN_LOGDIR)
    writer = tf.summary.create_file_writer(TRAIN_LOGDIR)

    trainset = Dataset('train')
    testset = Dataset('test')

    steps_per_epoch = len(trainset)
    global_steps = tf.Variable(1, trainable=False, dtype=tf.int64)
    warmup_steps = TRAIN_WARMUP_EPOCHS * steps_per_epoch
    total_steps = TRAIN_EPOCHS * steps_per_epoch

    if TRAIN_TRANSFER:
        Darknet = Create_Yolo(input_size=YOLO_INPUT_SIZE, CLASSES=YOLO_COCO_CLASSES)
        load_yolo_weights(Darknet, Darknet_weights) # use darknet weights

    yolo = Create_Yolo(input_size=YOLO_INPUT_SIZE, training=True, CLASSES=TRAIN_CLASSES)
    if TRAIN_FROM_CHECKPOINT:
        try:
            yolo.load_weights(f"./checkpoints/{TRAIN_MODEL_NAME}")
        except ValueError:
            print("Shapes are incompatible, transfering Darknet weights")
            TRAIN_FROM_CHECKPOINT = False

    if TRAIN_TRANSFER and not TRAIN_FROM_CHECKPOINT:
        for i, l in enumerate(Darknet.layers):
            layer_weights = l.get_weights()
            if layer_weights != []:
                try:
                    yolo.layers[i].set_weights(layer_weights)
                except:
                    print("skipping", yolo.layers[i].name)

    optimizer = tf.keras.optimizers.Adam()


    def train_step(image_data, target):
        with tf.GradientTape() as tape:
            pred_result = yolo(image_data, training=True)
            giou_loss=conf_loss=prob_loss=0

            # optimizing process
            grid = 3 if not TRAIN_YOLO_TINY else 2
            for i in range(grid):
                conv, pred = pred_result[i*2], pred_result[i*2+1]
                loss_items = compute_loss(pred, conv, *target[i], i, CLASSES=TRAIN_CLASSES)
                giou_loss += loss_items[0]
                conf_loss += loss_items[1]
                prob_loss += loss_items[2]

            total_loss = giou_loss + conf_loss + prob_loss

            gradients = tape.gradient(total_loss, yolo.trainable_variables)
            optimizer.apply_gradients(zip(gradients, yolo.trainable_variables))

            # update learning rate
            # about warmup: https://arxiv.org/pdf/1812.01187.pdf&usg=ALkJrhglKOPDjNt6SHGbphTHyMcT0cuMJg
            global_steps.assign_add(1)
            if global_steps < warmup_steps:# and not TRAIN_TRANSFER:
                lr = global_steps / warmup_steps * TRAIN_LR_INIT
            else:
                lr = TRAIN_LR_END + 0.5 * (TRAIN_LR_INIT - TRAIN_LR_END)*(
                    (1 + tf.cos((global_steps - warmup_steps) / (total_steps - warmup_steps) * np.pi)))
            optimizer.lr.assign(lr.numpy())

            # writing summary data
            with writer.as_default():
                tf.summary.scalar("lr", optimizer.lr, step=global_steps)
                tf.summary.scalar("loss/total_loss", total_loss, step=global_steps)
                tf.summary.scalar("loss/giou_loss", giou_loss, step=global_steps)
                tf.summary.scalar("loss/conf_loss", conf_loss, step=global_steps)
                tf.summary.scalar("loss/prob_loss", prob_loss, step=global_steps)
            writer.flush()

        return global_steps.numpy(), optimizer.lr.numpy(), giou_loss.numpy(), conf_loss.numpy(), prob_loss.numpy(), total_loss.numpy()

    validate_writer = tf.summary.create_file_writer(TRAIN_LOGDIR)
    def validate_step(image_data, target):
        with tf.GradientTape() as tape:
            pred_result = yolo(image_data, training=False)
            giou_loss=conf_loss=prob_loss=0

            # optimizing process
            grid = 3 if not TRAIN_YOLO_TINY else 2
            for i in range(grid):
                conv, pred = pred_result[i*2], pred_result[i*2+1]
                loss_items = compute_loss(pred, conv, *target[i], i, CLASSES=TRAIN_CLASSES)
                giou_loss += loss_items[0]
                conf_loss += loss_items[1]
                prob_loss += loss_items[2]

            total_loss = giou_loss + conf_loss + prob_loss

        return giou_loss.numpy(), conf_loss.numpy(), prob_loss.numpy(), total_loss.numpy()

    mAP_model = Create_Yolo(input_size=YOLO_INPUT_SIZE, CLASSES=TRAIN_CLASSES) # create second model to measure mAP

    best_val_loss = 1000 # should be large at start
    for epoch in range(TRAIN_EPOCHS):
        for image_data, target in trainset:
            results = train_step(image_data, target)
            cur_step = results[0]%steps_per_epoch
            print("epoch:{:2.0f} step:{:5.0f}/{}, lr:{:.6f}, giou_loss:{:7.2f}, conf_loss:{:7.2f}, prob_loss:{:7.2f}, total_loss:{:7.2f}"
                  .format(epoch, cur_step, steps_per_epoch, results[1], results[2], results[3], results[4], results[5]))

        if len(testset) == 0:
            print("configure TEST options to validate model")
            yolo.save_weights(os.path.join(TRAIN_CHECKPOINTS_FOLDER, TRAIN_MODEL_NAME))
            continue

        count, giou_val, conf_val, prob_val, total_val = 0., 0, 0, 0, 0
        for image_data, target in testset:
            results = validate_step(image_data, target)
            count += 1
            giou_val += results[0]
            conf_val += results[1]
            prob_val += results[2]
            total_val += results[3]
        # writing validate summary data
        with validate_writer.as_default():
            tf.summary.scalar("validate_loss/total_val", total_val/count, step=epoch)
            tf.summary.scalar("validate_loss/giou_val", giou_val/count, step=epoch)
            tf.summary.scalar("validate_loss/conf_val", conf_val/count, step=epoch)
            tf.summary.scalar("validate_loss/prob_val", prob_val/count, step=epoch)
        validate_writer.flush()

        print("\n\ngiou_val_loss:{:7.2f}, conf_val_loss:{:7.2f}, prob_val_loss:{:7.2f}, total_val_loss:{:7.2f}\n\n".
              format(giou_val/count, conf_val/count, prob_val/count, total_val/count))

        if TRAIN_SAVE_CHECKPOINT and not TRAIN_SAVE_BEST_ONLY:
            save_directory = os.path.join(TRAIN_CHECKPOINTS_FOLDER, TRAIN_MODEL_NAME+"_val_loss_{:7.2f}".format(total_val/count))
            yolo.save_weights(save_directory)
        if TRAIN_SAVE_BEST_ONLY and best_val_loss>total_val/count:
            save_directory = os.path.join(TRAIN_CHECKPOINTS_FOLDER, TRAIN_MODEL_NAME)
            yolo.save_weights(save_directory)
            best_val_loss = total_val/count
        if not TRAIN_SAVE_BEST_ONLY and not TRAIN_SAVE_CHECKPOINT:
            save_directory = os.path.join(TRAIN_CHECKPOINTS_FOLDER, TRAIN_MODEL_NAME)
            yolo.save_weights(save_directory)

    # measure mAP of trained custom model
        try:
            mAP_model.load_weights(save_directory) # use keras weights
            get_mAP(mAP_model, testset, score_threshold=TEST_SCORE_THRESHOLD, iou_threshold=TEST_IOU_THRESHOLD)
        except UnboundLocalError:
            print("You don't have saved model weights to measure mAP, check TRAIN_SAVE_BEST_ONLY and TRAIN_SAVE_CHECKPOINT lines in configs.py")
try:
    if __name__ == '__main__':
        main()
except Exception as ex:
    print(ex)
    pass

GPUs [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
'cp949' codec can't decode byte 0xed in position 3: illegal multibyte sequence


In [15]:
f = open('E:/air_train_.txt', 'a', encoding='utf-8')
for i in x_train:
    f.write(i+'\n')
f.close()
f = open('E:/air_val_.txt', 'a', encoding='utf-8')
for i in x_val:
    f.write(i+'\n')
f.close()

In [38]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [39]:
tf.test.gpu_device_name()


'/device:GPU:0'

In [11]:
aa = open('D:/yolov4_TF/data/air/new_name.names', 'a', encoding='utf-8')
with open('D:/yolov4_TF/data/air/class_name.names', encoding='utf-8') as f:
    for i in f:
        # f.write(i+ '\n')
        aa.write(i.split('\n')[0]+'\n')
aa.close()